In [2]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import FloatType

spark=SparkSession.builder.appName('FordGoBike').getOrCreate()
spark

23/05/09 08:48:20 WARN Utils: Your hostname, codespaces-3e9f4e resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
23/05/09 08:48:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/09 08:48:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/09 08:48:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
path="/workspaces/develhope-Data5-Team3/Data/FordGoBike"
df_pyspark=spark.read.csv(path,header=True, inferSchema=True)

In [4]:
df_pyspark = df_pyspark.withColumn('start_time_timestamp', F.to_timestamp(F.concat_ws(':', 'start time hour', 'start time minute', 'start time seconds')))\
                        .withColumn('end_time_timestamp', F.to_timestamp(F.concat_ws(':', 'end_time hour', 'end_time minute', 'end_time seconds')))

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `start time hour` cannot be resolved. Did you mean one of the following? [`start_time`, `start_am_pm`, `start_station_id`, `end_time`, `start_station_name`].;
'Project [start_time#17, start_am_pm#18, end_time#19, end_am_pm#20, start_station_id#21, start_station_name#22, start_station_latitude#23, start_station_longitude#24, end_station_id#25, end_station_name#26, end_station_latitude#27, end_station_longitude#28, bike_id#29, user_type#30, member_birth_year#31, member_gender#32, pyment#33, haversine_distance#34, Diff_in_seconds#35, Diff_in_minutes#36, Trip_cost#37, to_timestamp(concat_ws(:, 'start time hour, 'start time minute, 'start time seconds), None, TimestampType, Some(Etc/UTC), false) AS start_time_timestamp#59]
+- Relation [start_time#17,start_am_pm#18,end_time#19,end_am_pm#20,start_station_id#21,start_station_name#22,start_station_latitude#23,start_station_longitude#24,end_station_id

In [5]:
df_pyspark=df_pyspark.withColumn('Diff_in_seconds',F.col('end_time').cast('long')-F.col('start_time').cast('long'))\
    .withColumn('Diff_in_minutes',(F.col('Diff_in_seconds')/60))\
    .withColumn('Trip_cost',(F.col('Diff_in_minutes')*0.35))

In [6]:
#df_pyspark=df_pyspark.drop(*['end_time_timestamp','end_time_timestamp'],)
df_pyspark.show()

+-------------------+-----------+-------------------+---------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+------------------+---------------+------------------+------------------+
|         start_time|start_am_pm|           end_time|end_am_pm|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|haversine_distance|Diff_in_seconds|   Diff_in_minutes|         Trip_cost|
+-------------------+-----------+-------------------+---------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+--------

In [8]:
df_pyspark.write.option("header",True).mode('overwrite').csv('/workspaces/develhope-Data5-Team3/Data/week_2')

In [ ]:
df_pyspark.groupBy("bike_id").agg(F.sum("haversine_distance").alias("sum_distance")).sort(F.desc("sum_distance")).show(10)